# SETUP

In [1]:
import os

# Set the environment variable 'TF_CPP_MIN_LOG_LEVEL' to '3'.
# This suppresses most of TensorFlow's logging output, keeping the console clean.
# Levels: 0 = all logs, 1 = no info, 2 = no warnings, 3 = no errors.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Set the environment variable 'TF_ENABLE_ONEDNN_OPTS' to '0'.
# This disables oneDNN/MKL optimizations in TensorFlow
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import math
import numpy as np
import matplotlib.pyplot as plt

import json

import keras_tuner as kt
import tensorflow as tf

from geexhp import datavis as dvis
dvis.configure_matplotlib()

# DATA PREPROCESSING PIPELINE

In [2]:
def compute_normalization_stats(train_tfrecord_path):
    # Initialize accumulators
    stats = {
        'inputs': {'UV': {'sum': 0., 'sq_sum': 0., 'count': 0},
                    'Vis': {'sum': 0., 'sq_sum': 0., 'count': 0},
                    'NIR': {'sum': 0., 'sq_sum': 0., 'count': 0}},
        'outputs': {key: {'sum': 0., 'sq_sum': 0., 'count': 0} 
                   for key in ['OBJECT-RADIUS-REL-EARTH', 'OBJECT-GRAVITY', 
                                'ATMOSPHERE-TEMPERATURE', 'ATMOSPHERE-PRESSURE',
                                'log_C2H6', 'log_CH4', 'log_CO', 'log_CO2', 'log_H2O', 'log_N2', 'log_N2O', 'log_O2', 'log_O3']}
    }

    # Parse function for raw data
    def parse_fn(example):
        features = {
            # Inputs
            # 'NOISY_ALBEDO_B-NIR': tf.io.VarLenFeature(tf.float32),
            # 'NOISY_ALBEDO_B-UV': tf.io.VarLenFeature(tf.float32),
            # 'NOISY_ALBEDO_B-Vis': tf.io.VarLenFeature(tf.float32),

            'ALBEDO_B-NIR': tf.io.VarLenFeature(tf.float32),
            'ALBEDO_B-UV': tf.io.VarLenFeature(tf.float32),
            'ALBEDO_B-Vis': tf.io.VarLenFeature(tf.float32),

            # Outputs
            'OBJECT-RADIUS-REL-EARTH': tf.io.FixedLenFeature([], tf.float32),
            'OBJECT-GRAVITY': tf.io.FixedLenFeature([], tf.float32),
            'ATMOSPHERE-TEMPERATURE': tf.io.FixedLenFeature([], tf.float32),
            'ATMOSPHERE-PRESSURE': tf.io.FixedLenFeature([], tf.float32),
            'log_C2H6': tf.io.FixedLenFeature([], tf.float32),
            'log_CH4': tf.io.FixedLenFeature([], tf.float32),
            'log_CO': tf.io.FixedLenFeature([], tf.float32),
            'log_CO2': tf.io.FixedLenFeature([], tf.float32),
            'log_H2O': tf.io.FixedLenFeature([], tf.float32),
            'log_N2': tf.io.FixedLenFeature([], tf.float32),
            'log_N2O': tf.io.FixedLenFeature([], tf.float32),
            'log_O2': tf.io.FixedLenFeature([], tf.float32),
            'log_O3': tf.io.FixedLenFeature([], tf.float32)
        }
        return tf.io.parse_single_example(example, features)

    # Process dataset
    dataset = tf.data.TFRecordDataset(train_tfrecord_path)
    dataset = dataset.map(parse_fn)
    
    for batch in dataset.batch(1000):  # Process in chunks
        # Inputs
        for region in ['UV', 'Vis', 'NIR']:
            key = f'ALBEDO_B-{region}'
            data = tf.sparse.to_dense(batch[key]).numpy()
            stats['inputs'][region]['sum'] += np.sum(data)
            stats['inputs'][region]['sq_sum'] += np.sum(data**2)
            stats['inputs'][region]['count'] += data.size
            
        # Outputs 
        for key in stats['outputs']:
            data = batch[key].numpy()
            stats['outputs'][key]['sum'] += np.sum(data)
            stats['outputs'][key]['sq_sum'] += np.sum(data**2)
            stats['outputs'][key]['count'] += data.size

    # Calculate final stats
    final_stats = {}
    
    # Input stats
    final_stats['inputs'] = {}
    for region in ['UV', 'Vis', 'NIR']:
        mean = stats['inputs'][region]['sum'] / stats['inputs'][region]['count']
        std = np.sqrt((stats['inputs'][region]['sq_sum'] / stats['inputs'][region]['count']) - mean**2)
        final_stats['inputs'][region] = {'mean': float(mean), 'std': float(std)}
    
    # Output stats
    final_stats['outputs'] = {}
    for key in stats['outputs']:
        mean = stats['outputs'][key]['sum'] / stats['outputs'][key]['count']
        std = np.sqrt((stats['outputs'][key]['sq_sum'] / stats['outputs'][key]['count']) - mean**2)
        final_stats['outputs'][key] = {'mean': float(mean), 'std': float(std)}
    
    # Save to JSON
    with open('normalization_stats.json', 'w') as f:
        json.dump(final_stats, f)
        
    return final_stats

# Run this once on your training data
# stats = compute_normalization_stats("../data/train.tfrecord")

# DATA LOADING PIPELINE IMPLEMENTATION

In [3]:
# Load statistics from Phase 1
with open('normalization_stats.json') as f:
    stats = json.load(f)

# Create lookup dictionaries for TF operations
input_stats = {
    'UV': (stats['inputs']['UV']['mean'], stats['inputs']['UV']['std']),
    'Vis': (stats['inputs']['Vis']['mean'], stats['inputs']['Vis']['std']),
    'NIR': (stats['inputs']['NIR']['mean'], stats['inputs']['NIR']['std'])
}

output_stats = {
    key: (stats['outputs'][key]['mean'], stats['outputs'][key]['std'])
    for key in stats['outputs']
}

In [4]:
def parse_example(example_proto, input_stats, output_stats):
    """
    Parse and normalize each TFRecord example.
    """
    raw_input_features = {
        'NOISY_ALBEDO_B-NIR': tf.io.VarLenFeature(tf.float32),
        'NOISY_ALBEDO_B-UV':  tf.io.VarLenFeature(tf.float32),
        'NOISY_ALBEDO_B-Vis': tf.io.VarLenFeature(tf.float32),
    }

    raw_output_features = {
        "OBJECT-RADIUS-REL-EARTH": tf.io.FixedLenFeature([], tf.float32),
        "OBJECT-GRAVITY":          tf.io.FixedLenFeature([], tf.float32),
        "ATMOSPHERE-TEMPERATURE":  tf.io.FixedLenFeature([], tf.float32),
        "ATMOSPHERE-PRESSURE":     tf.io.FixedLenFeature([], tf.float32),

        'log_C2H6': tf.io.FixedLenFeature([], tf.float32),
        'log_CH4':  tf.io.FixedLenFeature([], tf.float32),
        'log_CO':   tf.io.FixedLenFeature([], tf.float32),
        'log_CO2':  tf.io.FixedLenFeature([], tf.float32),
        'log_H2O':  tf.io.FixedLenFeature([], tf.float32),
        'log_N2':   tf.io.FixedLenFeature([], tf.float32),
        'log_N2O':  tf.io.FixedLenFeature([], tf.float32),
        'log_O2':   tf.io.FixedLenFeature([], tf.float32),
        'log_O3':   tf.io.FixedLenFeature([], tf.float32)
    }

    all_features = {**raw_input_features, **raw_output_features}
    parsed = tf.io.parse_single_example(example_proto, all_features)

    # ----------------------
    # 1) Normalize inputs
    # ----------------------
    # UV shape = [8], Vis shape = [94], NIR shape = [49]
    uv_spectrum  = tf.sparse.to_dense(parsed['NOISY_ALBEDO_B-UV'],  default_value=0.0)
    vis_spectrum = tf.sparse.to_dense(parsed['NOISY_ALBEDO_B-Vis'], default_value=0.0)
    nir_spectrum = tf.sparse.to_dense(parsed['NOISY_ALBEDO_B-NIR'], default_value=0.0)

    # Subtract mean, divide by std
    uv_mean,  uv_std  = input_stats['UV']
    vis_mean, vis_std = input_stats['Vis']
    nir_mean, nir_std = input_stats['NIR']

    uv_norm  = (uv_spectrum  - uv_mean)  / (uv_std)
    vis_norm = (vis_spectrum - vis_mean) / (vis_std)
    nir_norm = (nir_spectrum - nir_mean) / (nir_std)

    # Concatenate in ascending wavelength order [UV, Vis, NIR]
    # Resulting shape: (8 + 94 + 49) = 151
    full_spectrum = tf.concat([uv_norm, vis_norm, nir_norm], axis=0)

    # Expand dims to get shape (151, 1) for 1D CNN
    full_spectrum = tf.reshape(full_spectrum, [-1, 1])

    # ----------------------
    # 2) Normalize outputs
    # ----------------------
    processed_outputs = {}
    for key in output_stats:
        mean_val, std_val = output_stats[key]
        raw_val = parsed[key]
        processed_outputs[key] = (raw_val - mean_val) / (std_val)

    return full_spectrum, processed_outputs

In [5]:
def read_tfrecord(
    file_path, 
    input_stats, 
    output_stats, 
    batch_size=150, 
    shuffle_buffer=None,
    repeat=False
):
    dataset = tf.data.TFRecordDataset(file_path, num_parallel_reads=tf.data.AUTOTUNE)

    if repeat:
        dataset = dataset.repeat()  # indefinite repeat

    if shuffle_buffer:
        dataset = dataset.shuffle(shuffle_buffer)

    parsed_dataset = dataset.map(
        lambda x: parse_example(x, input_stats, output_stats), 
        num_parallel_calls=tf.data.AUTOTUNE
    )

    parsed_dataset = parsed_dataset.batch(batch_size)
    parsed_dataset = parsed_dataset.prefetch(tf.data.AUTOTUNE)

    return parsed_dataset

In [6]:
# The number of training samples in the dataset.
TRAIN_SAMPLES = 779887

# THe number of validation samples in the dataset.
VAL_SAMPLES = 66676

# Define the batch size to use for training, validation, and testing.
# A batch size of 32 means that the model will process 32 samples at a time.
BATCH_SIZE = 150

steps_per_epoch = math.ceil(TRAIN_SAMPLES / BATCH_SIZE)
validation_steps = math.ceil(VAL_SAMPLES / BATCH_SIZE)

In [7]:
train_ds = read_tfrecord("../data/train.tfrecord", input_stats, output_stats, shuffle_buffer=10000, repeat=True)
val_ds = read_tfrecord("../data/val.tfrecord", input_stats, output_stats)
test_ds = read_tfrecord("../data/test.tfrecord", input_stats, output_stats)

# Inspect a batch
sample_inputs, sample_outputs = next(iter(train_ds))

print("\n## Input Shapes:\n")
print(f"> {sample_inputs.shape}") 

print("\n## Output Ranges (standardized):\n")
for k,v in sample_outputs.items():
    print(f"> {k}: {tf.reduce_mean(v):.3f} ± {tf.math.reduce_std(v):.3f}")


## Input Shapes:

> (150, 151, 1)

## Output Ranges (standardized):

> OBJECT-RADIUS-REL-EARTH: 0.104 ± 1.006
> OBJECT-GRAVITY: 0.111 ± 1.019
> ATMOSPHERE-TEMPERATURE: 0.034 ± 1.045
> ATMOSPHERE-PRESSURE: 0.106 ± 1.043
> log_C2H6: 0.694 ± 0.000
> log_CH4: -1.244 ± 0.508
> log_CO: -0.823 ± 0.571
> log_CO2: -0.972 ± 0.756
> log_H2O: -0.306 ± 0.777
> log_N2: -0.679 ± 1.477
> log_N2O: -0.440 ± 0.233
> log_O2: 0.293 ± 0.498
> log_O3: -0.439 ± 0.226


In [8]:
# def count_samples(dataset: tf.data.Dataset):
#     """
#     Count the total number of samples in a dataset by iterating once.
#     If the dataset is batched, unbatch it first.
#     """
#     count = 0
#     for _ in dataset.unbatch():
#         count += 1
#     return count

# TRAIN_SAMPLES = count_samples(train_ds) # 779887
# print("Train samples:", TRAIN_SAMPLES)      

# VAL_SAMPLES = count_samples(val_ds)
# print("Validation samples:", VAL_SAMPLES)  # 66676

# MODEL ARCHITECTURE DESIGN

In [9]:
def build_model():
    """
    Build a multi-output 1D CNN model that processes the entire spectrum (UV + Vis + NIR).
    """
    # List of all output variables
    outputs_list = [
        "OBJECT-RADIUS-REL-EARTH",
        "OBJECT-GRAVITY",
        "ATMOSPHERE-TEMPERATURE",
        "ATMOSPHERE-PRESSURE",
        'log_C2H6',
        'log_CH4',
        'log_CO',
        'log_CO2',
        'log_H2O',
        'log_N2',
        'log_N2O',
        'log_O2',
        'log_O3'
    ]

    l2_reg = tf.keras.regularizers.l2(1e-4)

    # Single input: entire concatenated spectrum of shape (151, 1)
    input_spectrum = tf.keras.layers.Input(shape=(151, 1), name='full_spectrum')

    # --------------------------------------------
    # Residual block with optional pooling
    # --------------------------------------------
    def residual_block(x, filters, kernel_size=3, pool_size=2, strides=2):
        """
        Single residual block with a skip connection.
        """
        # Conv path 1
        x_init = tf.keras.layers.Conv1D(
            filters, 3, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2_reg
        )(x)
        x_init = tf.keras.layers.BatchNormalization()(x_init)
        x_init = tf.keras.layers.Activation('swish')(x_init)

        # Conv path 2
        x_main = tf.keras.layers.Conv1D(
            filters, kernel_size, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2_reg
        )(x_init)
        x_main = tf.keras.layers.BatchNormalization()(x_main)

        # Residual addition
        x_add = tf.keras.layers.Add()([x_init, x_main])
        x_out = tf.keras.layers.Activation('swish')(x_add)

        # Average pooling for downsampling (if strides>1)
        x_out = tf.keras.layers.AveragePooling1D(
            pool_size=pool_size, 
            strides=strides,  
            padding='same'
        )(x_out)
        return x_out

    # --------------------------------------------
    # Build a stack of residual blocks
    # --------------------------------------------
    x = input_spectrum
    x = residual_block(x, filters=16, kernel_size=3, pool_size=2, strides=2)   # output length ~ 151/2
    x = residual_block(x, filters=32, kernel_size=3, pool_size=2, strides=2)   # output length ~ 151/4
    x = residual_block(x, filters=64, kernel_size=5, pool_size=2, strides=2)   # output length ~ 151/8
    x = residual_block(x, filters=128, kernel_size=5, pool_size=2, strides=2)  # output length ~ 151/16

    # --------------------------------------------
    # Squeeze-and-Excite block (optional)
    # --------------------------------------------
    def squeeze_excite(block_input, ratio=8):
        filters = block_input.shape[-1]
        se = tf.keras.layers.GlobalAveragePooling1D()(block_input)
        se = tf.keras.layers.Dense(filters//ratio, activation='swish', kernel_regularizer=l2_reg)(se)
        se = tf.keras.layers.Dense(filters, activation='sigmoid', kernel_regularizer=l2_reg)(se)
        return tf.keras.layers.Multiply()([block_input, se])

    x = squeeze_excite(x)

    # --------------------------------------------
    # Global Pool or Flatten -> Dense layers
    # --------------------------------------------
    # Now reduce over the sequence dimension to get a single vector per sample
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    # Alternatively: x = tf.keras.layers.Flatten()(x)

    x = tf.keras.layers.Dense(256, activation='swish', kernel_regularizer=l2_reg)(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Dense(128, activation='swish', kernel_regularizer=l2_reg)(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # --------------------------------------------
    # Build separate output heads
    # --------------------------------------------
    outputs = {}
    for out_var in outputs_list:
        head = tf.keras.layers.Dense(64, activation='swish', kernel_regularizer=l2_reg)(x)
        head = tf.keras.layers.Dropout(0.3)(head)
        outputs[out_var] = tf.keras.layers.Dense(1, name=out_var)(head)

    # --------------------------------------------
    # Construct the Model
    # --------------------------------------------
    model = tf.keras.Model(inputs=input_spectrum, outputs=outputs)

    # Define MSE loss for each target
    losses = {name: tf.keras.losses.MeanSquaredError() for name in outputs_list}

    # Optimizer: AdamW
    optimizer = tf.keras.optimizers.AdamW(
        learning_rate=1e-5,
        weight_decay=1e-5,
        global_clipnorm=1.0
    )

    model.compile(
        optimizer=optimizer,
        loss=losses,
    )

    return model

In [10]:
model = build_model()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ full_spectrum       │ (None, 151, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 151, 16)   │         64 │ full_spectrum[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 151, 16)   │         64 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 151, 16)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 151, 16)   │        784 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 151, 16)   │         64 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 151, 16)   │          0 │ activation[0][0], │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 151, 16)   │          0 │ add[0][0]         │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling1d   │ (None, 76, 16)    │          0 │ activation_1[0][… │
│ (AveragePooling1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 76, 32)    │      1,568 │ average_pooling1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 76, 32)    │        128 │ conv1d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 76, 32)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 76, 32)    │      3,104 │ activation_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 76, 32)    │        128 │ conv1d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 76, 32)    │          0 │ activation_2[0][… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 76, 32)    │          0 │ add_1[0][0]       │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling1d_1 │ (None, 38, 32)    │          0 │ activation_3[0][… │
│ (AveragePooling1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 38, 64)    │      6,208 │ average_pooling1… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 320,813 (1.22 MB)

 Trainable params: 319,085 (1.22 MB)

 Non-trainable params: 1,728 (6.75 KB)

# TRAINING STRATEGY & OPTIMIZATION

In [11]:
EPOCHS = 5

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        min_delta=0.001,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7
    ),

]

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    #steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    #validation_steps=validation_steps
)

Epoch 1/5
 904596/Unknown 75759s 84ms/step - ATMOSPHERE-PRESSURE_loss: 0.5212 - ATMOSPHERE-TEMPERATURE_loss: 0.9247 - OBJECT-GRAVITY_loss: 0.5101 - OBJECT-RADIUS-REL-EARTH_loss: 0.5004 - log_C2H6_loss: 0.2258 - log_CH4_loss: 0.3504 - log_CO2_loss: 0.5360 - log_CO_loss: 0.4414 - log_H2O_loss: 0.4526 - log_N2O_loss: 0.2357 - log_N2_loss: 0.6934 - log_O2_loss: 0.3757 - log_O3_loss: 0.2339 - loss: 6.1848

KeyboardInterrupt: 

In [ ]:
predictions = model.predict(test_ds) 

In [ ]:
# Collect ground-truth (normalized) from test_ds
all_true = {key: [] for key in output_stats.keys()}
for batch_in, batch_out in test_ds:
    for key in output_stats:
        all_true[key].append(batch_out[key].numpy())
for key in all_true:
    all_true[key] = np.concatenate(all_true[key], axis=0)

In [ ]:
# Denormalize
denorm_pred = {}
denorm_true = {}
for key in output_stats:
    mean_val = output_stats[key]['mean']
    std_val  = output_stats[key]['std']

    y_pred_norm = predictions[key].reshape(-1)
    y_true_norm = all_true[key].reshape(-1)

    # Denormalize
    denorm_pred[key] = y_pred_norm * std_val + mean_val
    denorm_true[key] = y_true_norm * std_val + mean_val

In [ ]:
for key in output_stats:
    plt.figure(figsize=(5,5))
    plt.scatter(denorm_true[key], denorm_pred[key], s=2, alpha=0.3)
    plt.xlabel(f"True {key}")
    plt.ylabel(f"Pred {key}")
    plt.title(f"True vs. Pred: {key}")
    min_val = min(denorm_true[key].min(), denorm_pred[key].min())
    max_val = max(denorm_true[key].max(), denorm_pred[key].max())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--')
    plt.show()